<a href="https://colab.research.google.com/github/Anunineleaps6/Capstone_Sales/blob/main/Capstone_sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [415]:
!pip install gspread oauth2client pandas numpy


In [416]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("/content/drive/MyDrive/CAPSTONE_FINAL/creds.json", scope)
client = gspread.authorize(creds)


In [417]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta


In [418]:
!pip install phonenumbers

In [419]:
import phonenumbers

def generate_phone(country, region_mapping):
    # Try to get the country code from the region_mapping first
    code = None
    for region, data in region_mapping.items():
        if country in data["countries"]:
            code = data["dial_code"].get(country)
            break

    if code:
        try:
            # Generate a random 10-digit number
            local_number = ''.join([str(random.randint(0, 9)) for _ in range(10)])
            # Attempt to parse and format using phonenumbers
            # Note: This might not create *valid* numbers, but will follow the format
            # We need a country code without the '+' for the parse function
            country_code_for_parse = code.lstrip('+')
            # We need to find a region code (like US, GB, IN) for the parse function
            # This is a simplified approach; a more robust solution would map countries to region codes
            country_region_code = ""
            for region_key, data in region_mapping.items():
                if country in data.get("countries", []):
                     # This is a simplification; need a proper country to region code mapping
                     # For now, we can try to infer from common country codes or use a default
                     # A more accurate approach would involve a comprehensive country code to region code mapping
                     # Let's use a placeholder logic or rely on the dial code if a direct mapping is complex
                     pass # Will refine this

            # A better approach might be to directly format a random number with the known dial code
            # We'll stick to generating a random 10-digit number and prefixing the dial code
            # as phonenumbers.parse requires a known region or a leading '+' which we already have

            # Let's refine the phonenumbers usage or simplify if direct generation is sufficient
            # Given the user wants the format "+[country_code]-[10_digits]", direct generation is simpler
            number = ''.join([str(random.randint(0, 9)) for _ in range(10)])
            return f"{code}-{number}"

        except Exception as e:
            print(f"Error generating phone for {country} with code {code}: {e}")
            # Fallback to a generic format if phonenumbers fails
            return f"{code}-{random.randint(1000000000, 9999999999)}"
    else:
        # Fallback for countries not in region_mapping
        return f"+00-{random.randint(1000000000, 9999999999)}"

# Now, modify the function call in the lead generation loop in cell 9212e2f3
# (This modification will be done in the next modify_cells command)

In [420]:
#LEADS
import pandas as pd
import random
import uuid
from datetime import datetime, timedelta

# ------------------------------------------
# 1. Region, Country, Dial Code Mapping (Used for phone number generation)
# ------------------------------------------
region_mapping = {
    "NA": {"countries": ["USA", "Canada"], "dial_code": {"USA": "+1", "Canada": "+1"}, "industries": ["Technology", "Healthcare", "Finance", "Retail"], "services": ["Data", "Marketing", "Cloud"]},
    "EU": {"countries": ["UK", "Germany", "France", "Netherlands", "Switzerland", "Sweden", "Israel"], "dial_code": {"UK": "+44", "Germany": "+49", "France": "+33", "Netherlands": "+31", "Switzerland": "+41", "Sweden": "+46", "Israel": "+972"}, "industries": ["Automotive", "Pharmaceuticals", "Banking", "Consulting"], "services": ["Consulting", "Engineering", "Operations"]},
    "APAC": {"countries": ["India", "Japan", "Singapore", "China", "Indonesia", "Australia", "New Zealand", "S. Korea"], "dial_code": {"India": "+91", "Japan": "+81", "Singapore": "+65", "China": "+86", "Indonesia": "+62", "Australia": "+61", "New Zealand": "+64", "S. Korea": "+82"}, "industries": ["Technology", "E-Commerce", "Manufacturing", "Finance"], "services": ["AI", "Staffing", "T&M"]},
    "ME": {"countries": ["UAE", "Saudi Arabia", "Qatar", "Bahrain"], "dial_code": {"UAE": "+971", "Saudi Arabia": "+966", "Qatar": "+974", "Bahrain": "+973"}, "industries": ["Energy", "Real Estate", "Banking"], "services": ["Data", "Consulting"]},
    "LA": {"countries": ["Brazil", "Mexico", "Argentina", "Chile", "Colombia", "Peru"], "dial_code": {"Brazil": "+55", "Mexico": "+52", "Argentina": "+54", "Chile": "+56", "Colombia": "+57", "Peru": "+51"}, "industries": ["Manufacturing", "Agriculture", "Energy"], "services": ["Operations", "Logistics"]},
    "AF": {"countries": ["S. Africa", "Mozambique", "Kenya", "Morocco"], "dial_code": {"S. Africa": "+27", "Mozambique": "+258", "Kenya": "+254", "Morocco": "+212"}, "industries": ["Telecommunications", "Mining", "Finance"], "services": ["Staffing", "Service"]}
}

# ------------------------------------------
# 2. Predefined Lead Data (for Company_Name, Country, Region, Lead_ID) - 108 Companies
# ------------------------------------------
predefined_leads = [
    # North America (20)
    ("Uber", "USA", "NA", "LDUBR-NA-001"), ("Walmart", "USA", "NA", "LDWMT-NA-002"), ("Ford Motor Co", "USA", "NA", "LDFRD-NA-003"), ("Amazon", "USA", "NA", "LDAMZ-NA-004"),
    ("Microsoft", "USA", "NA", "LDMST-NA-005"), ("Target", "USA", "NA", "LDTGT-NA-006"), ("Uber Eats", "USA", "NA", "LDUBE-NA-007"), ("CVS Health", "USA", "NA", "LDCVS-NA-008"),
    ("McKesson", "USA", "NA", "LDMCK-NA-009"), ("TD Bank Group", "Canada", "NA", "LDTDB-NA-010"), ("Shopify", "Canada", "NA", "LDSHF-NA-011"), ("Suncor Energy", "Canada", "NA", "LDSCR-NA-012"),
    ("Kaiser Permanente", "USA", "NA", "LDKPR-NA-013"), ("Wells Fargo", "USA", "NA", "LDWLF-NA-014"), ("Canadian Tire", "Canada", "NA", "LDCTC-NA-015"), ("American Tower", "USA", "NA", "LDAMT-NA-016"),
    ("Tim Hortons", "Canada", "NA", "LDTHC-NA-017"), ("Lowe's", "USA", "NA", "LDLWS-NA-018"), ("PicnicHealth", "USA", "NA", "LDPCH-NA-019"), ("Credit Saison India (USA operations)", "USA", "NA", "LDCSI-NA-020"),

    # Europe (16)
    ("Tesco", "UK", "EU", "LDTSCO-EU-021"), ("Lufthansa Group", "Germany", "EU", "LDLFT-EU-022"), ("Allianz Direct", "Germany", "EU", "LDALZ-EU-023"), ("HSBC", "UK", "EU", "LDHSBC-EU-024"),
    ("Sage Group", "UK", "EU", "LDSGE-EU-025"), ("Schneider Electric", "France", "EU", "LDSNE-EU-026"), ("Philips Healthcare", "Netherlands", "EU", "LDPHL-EU-027"), ("Vodafone", "UK", "EU", "LDVDF-EU-028"),
    ("Siemens", "Germany", "EU", "LDSMN-EU-029"), ("Nestlé", "Switzerland", "EU", "LDNST-EU-030"), ("ING Group", "Netherlands", "EU", "LDING-EU-031"), ("ABB", "Switzerland", "EU", "LDABB-EU-032"),
    ("Volvo Group", "Sweden", "EU", "LDVLV-EU-033"), ("Carrefour", "France", "EU", "LDCFR-EU-034"), ("ZF Group", "Germany", "EU", "LDZFG-EU-035"), ("Trigo", "UK", "EU", "LDTRG-EU-036"), # Assuming UK operations for Trigo

    # Asia-Pacific (14)
    ("DBS Bank", "Singapore", "APAC", "LDDBS-APAC-037"), ("Kiwibank", "New Zealand", "APAC", "LDKWB-APAC-038"), ("Reliance Retail", "India", "APAC", "LDRLR-APAC-039"), ("Tata Consultancy Services", "India", "APAC", "LDTCS-APAC-040"),
    ("SoftBank", "Japan", "APAC", "LDSFB-APAC-041"), ("Tokopedia / GoTo", "Indonesia", "APAC", "LDTKP-APAC-042"), ("Telstra", "Australia", "APAC", "LDTSL-APAC-043"), ("Westpac", "Australia", "APAC", "LDWSP-APAC-044"),
    ("infosys", "India", "APAC", "LDINF-APAC-045"), ("Grab", "Singapore", "APAC", "LDGRB-APAC-046"), ("JioMart", "India", "APAC", "LDJMR-APAC-047"), ("Toyota", "Japan", "APAC", "LDTYT-APAC-048"),
    ("Sony", "Japan", "APAC", "LDSNY-APAC-049"), ("Samsung", "S. Korea", "APAC", "LDSMS-APAC-050"),

    # Middle East (4)
    ("Saudi Aramco", "Saudi Arabia", "ME", "LDSAR-ME-051"), ("ADNOC", "UAE", "ME", "LDADC-ME-052"), ("QatarEnergy", "Qatar", "ME", "LDQTE-ME-053"), ("Investcorp", "Bahrain", "ME", "LDINV-ME-054"),

    # Latin America (10)
    ("Petrobras", "Brazil", "LA", "LDPBR-LA-055"), ("América Móvil", "Mexico", "LA", "LDAMX-LA-056"), ("MercadoLibre", "Argentina", "LA", "LDMCL-LA-057"), ("Codelco", "Chile", "LA", "LDCDL-LA-058"),
    ("Ecopetrol", "Colombia", "LA", "LDECP-LA-059"), ("Latam Airlines", "Chile", "LA", "LDLTM-LA-060"), ("Natura", "Brazil", "LA", "LDNTR-LA-061"), ("Itau Unibanco", "Brazil", "LA", "LDITU-LA-062"),
    ("Bancolombia", "Colombia", "LA", "LDBCL-LA-063"), ("Grupo Mexico", "Mexico", "LA", "LDGPM-LA-064"),

    # Africa (12)
    ("MTN Group", "S. Africa", "AF", "LDMTN-AF-065"), ("Sasol", "S. Africa", "AF", "LDSAL-AF-066"), ("Standard Bank Group", "S. Africa", "AF", "LDSBG-AF-067"), ("Vodacom", "S. Africa", "AF", "LDVDC-AF-068"),
    ("Safaricom", "Kenya", "AF", "LDSFC-AF-069"), ("KCB Group", "Kenya", "AF", "LDKCB-AF-070"), ("Attijariwafa Bank", "Morocco", "AF", "LDATJ-AF-071"), ("FirstRand", "S. Africa", "AF", "LDFRD-AF-072"),
    ("Absa Group", "S. Africa", "AF", "LDABG-AF-073"), ("Nedbank Group", "S. Africa", "AF", "LDNDB-AF-074"), ("Equity Group Holdings", "Kenya", "AF", "LDEGH-AF-075"), ("Telkom", "S. Africa", "AF", "LDTLK-AF-076"),

    # Adding more predefined companies to reach 108, maintaining the pattern and region assignment
    ("Infosys", "India", "APAC", "LDINF-APAC-077"), ("Wipro", "India", "APAC", "LDWIP-APAC-078"), ("HCL Technologies", "India", "APAC", "LDHCL-APAC-079"),
    ("Tech Mahindra", "India", "APAC", "LDTMH-APAC-080"), ("Larsen & Toubro Infotech", "India", "APAC", "LDLTI-APAC-081"), ("Mindtree", "India", "APAC", "LDMNT-APAC-082"),
    ("Mphasis", "India", "APAC", "LDMPH-APAC-083"), ("L&T Technology Services", "India", "APAC", "LDLTTS-APAC-084"), ("Persistent Systems", "India", "APAC", "LDPST-APAC-085"),
    ("Coforge", "India", "APAC", "LDCFG-APAC-086"), ("Zensar Technologies", "India", "APAC", "LDZNR-APAC-087"), ("Firstsource Solutions", "India", "APAC", "LDFRS-APAC-088"),
    ("ExlService Holdings", "India", "APAC", "LDEXL-APAC-089"), ("Genus Power Infrastructures", "India", "APAC", "LDGPI-APAC-090"), ("Cyient", "India", "APAC", "LDCYI-APAC-091"),

    ("Tata Motors", "India", "APAC", "LDTMO-APAC-092"), ("Mahindra & Mahindra", "India", "APAC", "LDMAM-APAC-093"), ("Maruti Suzuki India", "India", "APAC", "LDMSI-APAC-094"),
    ("Hyundai Motor India", "India", "APAC", "LDHMI-APAC-095"), ("Kia India", "India", "APAC", "LDKIA-APAC-096"), ("Toyota Kirloskar Motor", "India", "APAC", "LDTKM-APAC-097"),
    ("Honda Cars India", "India", "APAC", "LDHCI-APAC-098"), ("Renault India", "India", "APAC", "LDRNI-APAC-099"), ("Nissan Motor India", "India", "APAC", "LDNMI-APAC-100"),

    ("Apple", "USA", "NA", "LDAPL-NA-101"), ("Google", "USA", "NA", "LDGOO-NA-102"), ("Facebook", "USA", "NA", "LDFAB-NA-103"), ("Netflix", "USA", "NA", "LDNFL-NA-104"),
    ("Salesforce", "USA", "NA", "LDSFC-NA-105"), ("Tesla", "USA", "NA", "LDTES-NA-106"), ("IBM", "USA", "NA", "LDIBM-NA-107"), ("Intel", "USA", "NA", "LDINT-NA-108")
]


# ------------------------------------------
# 3. Generate Sales Reps (Unevenly distributed based on region size)
# ------------------------------------------
rep_names = [
    "Alice Smith", "Bob Johnson", "Charlie Lee", "Diana King", "Edward Brown",
    "Fiona Green", "George White", "Hannah Black", "Ian Gray", "Jill Lewis",
    "Kevin Martin", "Laura Scott", "Mike Turner", "Nina Adams", "Oscar Clark",
    "Paula Young", "Quinn Hall", "Rachel Allen", "Steve Wright", "Tina Baker"
] # 20 names

# Distribute 20 reps unevenly across regions (adjust numbers as needed to total 20)
# Example: NA: 4, EU: 4, APAC: 5, ME: 2, LA: 3, AF: 2
region_rep_distribution = {"NA": 4, "EU": 4, "APAC": 5, "ME": 2, "LA": 3, "AF": 2}

rep_list = []
rep_name_index = 0
for region, count in region_rep_distribution.items():
    for i in range(count):
        full_name = rep_names[rep_name_index]
        initials = "".join([part[0] for part in full_name.split()]).upper()
        rep_id = f"{initials}_{region}"
        rep_list.append({"rep_id": rep_id, "rep_name": full_name, "region": region})
        rep_name_index += 1

# Convert to DataFrame
rep_df = pd.DataFrame(rep_list)

# ------------------------------------------
# 4. Generate Leads DataFrame
# ------------------------------------------
def get_company_email(name):
    # Basic email domain generation
    domain = name.lower().replace(" ", "").replace("&", "").replace(".", "").replace("/", "").replace("(", "").replace(")", "") + ".com"
    return domain

# Use the generate_phone function defined in the previous cell (ad0fa588)

def generate_timestamp(start_days_ago=730): # Up to 2 years ago
    created = datetime.now() - timedelta(days=random.randint(0, start_days_ago), minutes=random.randint(0, 1440), seconds=random.randint(0, 86400))
    updated = created + timedelta(minutes=random.randint(10, 10000)) # Updated is after created
    return created.strftime("%Y-%m-%d %H:%M:%S"), updated.strftime("%Y-%m-%d %H:%M:%S")

def get_company_initials(name):
    return "".join([w[0] for w in name.split()]).upper()

# Define service categories and industries based on region_mapping for random lead generation
all_service_categories = list(set([service for region_data in region_mapping.values() for service in region_data.get('services', [])]))
all_industries = list(set([industry for region_data in region_mapping.values() for industry in region_data.get('industries', [])]))


# Generate a list of more descriptive company name parts for random leads
company_name_parts = {
    "adjectives": ["Global", "Strategic", "Innovative", "Dynamic", "Future", "Leading", "Creative", "Advanced"],
    "nouns": ["Solutions", "Systems", "Analytics", "Consulting", "Tech", "Ventures", "Partners", "Industries"]
}

def generate_random_company_name(adjectives, nouns):
    return f"{random.choice(adjectives)}{random.choice(nouns)}"

def get_abbreviated_company_name(name):
    # Simple abbreviation logic
    words = name.split()
    if len(words) > 1:
        return "".join([w[:2] for w in words]).upper() # Use first two letters of each word
    else:
        return name[:4].upper() # Use first four letters if single word


leads_data = []

# Add predefined leads
for company_name, country, region, lead_id in predefined_leads:
    company_email_domain = get_company_email(company_name)
    company_id = f"CMP_{get_company_initials(company_name)}"

    created_date, last_updated = generate_timestamp()
    deal_source = random.choices(["Inbound", "Outbound"], weights=[0.2, 0.8])[0]
    phone = generate_phone(country, region_mapping) # Use the new generate_phone function

    # Select a rep from same region
    region_reps = rep_df[rep_df["region"] == region]
    if not region_reps.empty:
        rep_row = region_reps.sample(n=1).iloc[0]
        assigned_rep_id = rep_row["rep_id"]
        rep_name = rep_row["rep_name"] # Keep rep_name for potential use
    else:
        # Handle case where no rep is available for the region
        assigned_rep_id = "Unassigned"
        rep_name = "Unassigned"

    lead_email = f"contact_{random.randint(1,100)}@{company_email_domain}"

    # Assign a service category and industry for predefined leads based on their country's typical services/industries
    # Find the region for the current country
    current_region = None
    for r, data in region_mapping.items():
        if country in data["countries"]:
            current_region = r
            break

    service_category = random.choice(region_mapping.get(current_region, {}).get('services', all_service_categories)) if current_region else random.choice(all_service_categories)
    company_industry = random.choice(region_mapping.get(current_region, {}).get('industries', all_industries)) if current_region else random.choice(all_industries)


    leads_data.append({
        "Lead_ID": lead_id,
        "Lead_Name": f"{company_name} Lead",
        "Company_Name": company_name,
        "Source": deal_source,
        "Region_ID": region,
        "Rep_ID": assigned_rep_id,
        "Created_Date": created_date,
        "Current_Stage": random.choice(["Lead Captured", "Qualified", "Contacted", "Demo Scheduled", "Proposal Sent", "Negotiation", "Deal Won", "Deal Lost"]),
        "Email": lead_email,
        "Phone": phone,
        "Priority": random.choice(["High", "Medium", "Low"]),
        "Last_Updated": last_updated,
        "Deal_Source": deal_source,
        "Company_ID": company_id,
        "Engagement_Level": random.choice(["High", "Medium", "Low"]),
        "Follow_Up_Threshold": random.choice([3, 5, 7, 10]),
        "Project_Term": random.choice(["0-6 months", "6-12 months", "12-24 months", ">24 months"]),
        "Budget_Tier": random.choice(["20-50L", "50-100L", "100L+"]),
        "Country": country,
        "Company_Industry": company_industry,
        "Service_Category": service_category
    })

# Generate additional random leads to reach 10,000
num_predefined_leads = len(predefined_leads)
num_additional_leads = 10000 - num_predefined_leads

# Get lists of all possible values for random generation
all_countries = [country for region_data in region_mapping.values() for country in region_data["countries"]]
all_regions = list(region_mapping.keys())

for i in range(num_additional_leads):
    company_name = generate_random_company_name(company_name_parts["adjectives"], company_name_parts["nouns"])
    country = random.choice(all_countries)
    # Find the region for the selected country using region_mapping
    region = next(r for r, data in region_mapping.items() if country in data["countries"])


    # Generate Lead ID in the desired format for random leads
    company_abbreviation = get_abbreviated_company_name(company_name)
    lead_id = f"LD{company_abbreviation}-{region}-{i+1:03}" # New format for random Lead IDs

    company_email_domain = get_company_email(company_name)
    company_id = f"CMP_{get_company_initials(company_name)}_{i+1:04}"

    created_date, last_updated = generate_timestamp()
    deal_source = random.choices(["Inbound", "Outbound"], weights=[0.2, 0.8])[0]
    phone = generate_phone(country, region_mapping) # Use the new generate_phone function

    # Select a rep from same region
    region_reps = rep_df[rep_df["region"] == region]
    if not region_reps.empty:
        rep_row = region_reps.sample(n=1).iloc[0]
        assigned_rep_id = rep_row["rep_id"]
        rep_name = rep_row["rep_name"]
    else:
        # Handle case where no rep is available for the region
        assigned_rep_id = "Unassigned"
        rep_name = "Unassigned"

    lead_email = f"contact_{random.randint(1,100)}@{company_email_domain}"

    # Assign a service category and industry for random leads based on the selected country's focus
    # Find the region for the current country
    current_region = None
    for r, data in region_mapping.items():
        if country in data["countries"]:
            current_region = r
            break

    service_category = random.choice(region_mapping.get(current_region, {}).get('services', all_service_categories)) if current_region else random.choice(all_service_categories)
    company_industry = random.choice(region_mapping.get(current_region, {}).get('industries', all_industries)) if current_region else random.choice(all_industries)


    # Generate other random data, potentially influenced by service_category if needed
    # (Currently, the random generation is independent of service_category,
    #  but can be made dependent here if specific correlations are desired)
    engagement_level = random.choice(["High", "Medium", "Low"])
    follow_up_threshold = random.choice([3, 5, 7, 10])
    project_term = random.choice(["0-6 months", "6-12 months", "12-24 months", ">24 months"])
    budget_tier = random.choice(["20-50L", "50-100L", "100L+"])
    current_stage = random.choice(["Lead Captured", "Qualified", "Contacted", "Demo Scheduled", "Proposal Sent", "Negotiation", "Deal Won", "Deal Lost"])
    priority = random.choice(["High", "Medium", "Low"])


    leads_data.append({
        "Lead_ID": lead_id, # Use the new formatted Lead ID
        "Lead_Name": f"{company_name} Lead",
        "Company_Name": company_name,
        "Source": deal_source, # Changed from "Lead Source" to "Source" to match requested column names
        "Region_ID": region,
        "Rep_ID": assigned_rep_id,
        "Created_Date": created_date,
        "Current_Stage": current_stage,
        "Email": lead_email,
        "Phone": phone,
        "Priority": priority,
        "Last_Updated": last_updated,
        "Deal_Source": deal_source,
        "Company_ID": company_id,
        "Engagement_Level": engagement_level,
        "Follow_Up_Threshold": follow_up_threshold,
        "Project_Term": project_term,
        "Budget_Tier": budget_tier,
        "Country": country,
        "Company_Industry": company_industry,
        "Service_Category": service_category
    })


df_lead = pd.DataFrame(leads_data)

# Reorder columns to match the requested order plus new columns
df_lead = df_lead[[
    "Lead_ID", "Lead_Name", "Company_Name", "Source", "Region_ID", "Rep_ID",
    "Created_Date", "Current_Stage", "Email", "Phone", "Priority", "Last_Updated",
    "Deal_Source", "Company_ID", "Engagement_Level", "Follow_Up_Threshold",
    "Project_Term", "Budget_Tier", "Country", "Company_Industry", "Service_Category"
]]


# After generating the initial df_lead, update the 'Current_Stage'
# to reflect the latest stage from the generated df_stage_history.
# This part needs to be executed after df_stage_history is generated in the next cell.
# This logic will be moved to a subsequent cell or integrated carefully if possible.


# ------------------------------------------
# 5. Export or View
# ------------------------------------------
# Save to CSV
# df_lead.to_csv("crm_leads_generated.csv", index=False)

# Display sample
display(df_lead.head())
display(df_lead.tail())
print(f"Generated {len(df_lead)} leads.")

,Lead_ID,Lead_Name,Company_Name,Source,Region_ID,Rep_ID,Created_Date,Current_Stage,Email,Phone,...,Last_Updated,Deal_Source,Company_ID,Engagement_Level,Follow_Up_Threshold,Project_Term,Budget_Tier,Country,Company_Industry,Service_Category
0,LDUBR-NA-001,Uber Lead,Uber,Outbound,NA,AS_NA,2024-07-30 17:33:27,Demo Scheduled,contact_70@uber.com,+1-2237976190,...,2024-08-06 13:00:27,Outbound,CMP_U,Medium,7,6-12 months,50-100L,USA,Retail,Marketing
1,LDWMT-NA-002,Walmart Lead,Walmart,Outbound,NA,BJ_NA,2024-11-14 20:50:13,Demo Scheduled,contact_47@walmart.com,+1-8466238837,...,2024-11-15 17:11:13,Outbound,CMP_W,Medium,3,6-12 months,20-50L,USA,Technology,Marketing
2,LDFRD-NA-003,Ford Motor Co Lead,Ford Motor Co,Outbound,NA,DK_NA,2023-11-16 17:05:59,Deal Lost,contact_61@fordmotorco.com,+1-1657048373,...,2023-11-17 08:53:59,Outbound,CMP_FMC,Low,5,12-24 months,20-50L,USA,Healthcare,Cloud
3,LDAMZ-NA-004,Amazon Lead,Amazon,Outbound,NA,BJ_NA,2024-08-14 06:24:18,Contacted,contact_15@amazon.com,+1-9338127286,...,2024-08-15 14:55:18,Outbound,CMP_A,High,5,>24 months,100L+,USA,Retail,Cloud
4,LDMST-NA-005,Microsoft Lead,Microsoft,Outbound,NA,CL_NA,2024-10-01 11:37:37,Demo Scheduled,contact_58@microsoft.com,+1-4400837397,...,2024-10-02 17:29:37,Outbound,CMP_M,High,3,6-12 months,50-100L,USA,Healthcare,Cloud


,Lead_ID,Lead_Name,Company_Name,Source,Region_ID,Rep_ID,Created_Date,Current_Stage,Email,Phone,...,Last_Updated,Deal_Source,Company_ID,Engagement_Level,Follow_Up_Threshold,Project_Term,Budget_Tier,Country,Company_Industry,Service_Category
9995,LDINNO-AF-9888,InnovativeSystems Lead,InnovativeSystems,Inbound,AF,SW_AF,2024-10-14 11:03:02,Negotiation,contact_69@innovativesystems.com,+258-5631770211,...,2024-10-16 05:24:02,Inbound,CMP_I_9888,High,7,0-6 months,50-100L,Mozambique,Telecommunications,Staffing
9996,LDFUTU-EU-9889,FutureTech Lead,FutureTech,Outbound,EU,GW_EU,2024-08-04 12:34:07,Proposal Sent,contact_92@futuretech.com,+972-1347982024,...,2024-08-08 23:41:07,Outbound,CMP_F_9889,High,10,>24 months,100L+,Israel,Consulting,Operations
9997,LDFUTU-NA-9890,FutureTech Lead,FutureTech,Outbound,NA,BJ_NA,2023-10-10 01:11:53,Proposal Sent,contact_33@futuretech.com,+1-2723807852,...,2023-10-11 23:58:53,Outbound,CMP_F_9890,High,5,6-12 months,50-100L,USA,Healthcare,Marketing
9998,LDINNO-ME-9891,InnovativeTech Lead,InnovativeTech,Outbound,ME,OC_ME,2025-07-09 08:09:08,Negotiation,contact_29@innovativetech.com,+971-6262904144,...,2025-07-11 15:02:08,Outbound,CMP_I_9891,Low,10,0-6 months,20-50L,UAE,Banking,Data
9999,LDCREA-AF-9892,CreativeVentures Lead,CreativeVentures,Outbound,AF,TB_AF,2024-08-12 08:58:32,Lead Captured,contact_49@creativeventures.com,+212-4618477631,...,2024-08-12 10:20:32,Outbound,CMP_C_9892,Low,7,6-12 months,20-50L,Morocco,Finance,Staffing


Generated 10000 leads.


In [421]:
#REGION
import pandas as pd
import numpy as np
import random

# Use unique Region_IDs from df_lead
region_ids = df_lead['Region_ID'].unique().tolist()
timezones = ["IST", "EST", "PST", "CET", "GMT"]
countries_in_leads = df_lead['Country'].unique().tolist() # Get countries from df_lead

region_data = []

# Define value mappings for company size for target calculation
company_size_value = {
    "Start-up": 1,
    "Small business": 5,
    "Medium Enterprise": 10,
    "Large Enterprise": 20
}

for region_id in region_ids:
    # Select a country that belongs to this region based on region_mapping
    country = random.choice([c for r, data in region_mapping.items() if r == region_id for c in data['countries'] if c in countries_in_leads])

    # Find reps assigned to this region
    reps_in_region = rep_df[rep_df['region'] == region_id]['rep_name'].tolist()
    region_head = random.choice(reps_in_region) if reps_in_region else "Unassigned"

    # Calculate Region_Target based on companies in the region
    companies_in_region = df_company[df_company['Country'].apply(lambda x: x in region_mapping.get(region_id, {}).get('countries', []))]

    # Calculate a score for each company based on size, ownership, funding, and interest area (simplified)
    # You can adjust weights and factors based on your desired logic
    company_scores = companies_in_region.apply(
        lambda row: company_size_value.get(row['Company_Size'], 0) *
                    (2 if row['Company_Ownership'] == 'Private Sector' else 1) *
                    (2 if row['Company_Funding'] in ['Series B', 'IPO'] else 1) *
                    (2 if row['Interest_Area'] in region_mapping.get(region_id, {}).get('services', []) else 1),
        axis=1
    )

    # Aggregate scores to get a raw region target
    raw_region_target = company_scores.sum()

    # Scale the raw target to a reasonable range (adjust scaling factor as needed)
    # Example: Scale by 100 to get targets in the thousands/millions
    region_target = round(raw_region_target * 500, 2) # Adjusted scaling factor

    region_data.append({
        "Region_ID": region_id,
        "Region_Name": f"Region_{region_id}",
        "Region_Head": region_head,
        "Timezone": random.choice(timezones),
        "Region_Target": region_target
    })

df_region = pd.DataFrame(region_data)
display(df_region.head())

,Region_ID,Region_Name,Region_Head,Timezone,Region_Target
0,NA,Region_NA,Diana King,PST,11915000
1,EU,Region_EU,Fiona Green,IST,35879000
2,APAC,Region_APAC,Jill Lewis,EST,40094000
3,ME,Region_ME,Nina Adams,IST,20346000
4,LA,Region_LA,Quinn Hall,EST,30702000


In [422]:
#SALES_REP
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# Use unique Rep_IDs and their corresponding regions from df_lead
sales_rep_data = []
added_rep_ids = set()

for index, row in df_lead.iterrows():
    rep_id = row['Rep_ID']
    region_id = row['Region_ID']
    # Add rep data only if not already added
    if rep_id not in added_rep_ids:
        # Find the rep name from the rep_df based on Rep_ID
        rep_info = rep_df[rep_df['rep_id'] == rep_id]
        rep_name = rep_info['rep_name'].iloc[0] if not rep_info.empty else f"Rep_{rep_id}"


        sales_rep_data.append({
            "Rep_ID": rep_id,
            "Rep_Name": rep_name, # Use name from rep_df
            "Email": f"{rep_id.lower()}@crmcorp.com", # Generate email based on Rep_ID
            "Phone": f"+91-9{random.randint(100000000,999999999)}", # Keep existing phone generation for now
            "Region_ID": region_id,
            "Join_Date": datetime(2020, 1, 1) + timedelta(days=random.randint(0, 1000)),
            "Current_Status": random.choice(["Active", "Inactive", "On Leave"]),
            "Performance_Rating": round(random.uniform(1.0, 5.0), 1),
            "Total_Deals_Closed": random.randint(0, 500),
            "Avg_conversion_time": round(random.uniform(3.0, 30.0), 2),
            "Win_rate": round(random.uniform(0.1, 0.9), 2)
        })
        added_rep_ids.add(rep_id)

# Add any reps from rep_df who are not assigned to any lead in df_lead
reps_not_in_leads = rep_df[~rep_df['rep_id'].isin(added_rep_ids)]
for index, rep_row in reps_not_in_leads.iterrows():
    rep_id = rep_row['rep_id']
    region_id = rep_row['region']
    rep_name = rep_row['rep_name']

    sales_rep_data.append({
        "Rep_ID": rep_id,
        "Rep_Name": rep_name,
        "Email": f"{rep_id.lower()}@crmcorp.com",
        "Phone": f"+91-9{random.randint(100000000,999999999)}",
        "Region_ID": region_id,
        "Join_Date": datetime(2020, 1, 1) + timedelta(days=random.randint(0, 1000)),
        "Current_Status": random.choice(["Active", "Inactive", "On Leave"]),
        "Performance_Rating": round(random.uniform(1.0, 5.0), 1),
        "Total_Deals_Closed": random.randint(0, 500),
        "Avg_conversion_time": round(random.uniform(3.0, 30.0), 2),
        "Win_rate": round(random.uniform(0.1, 0.9), 2)
    })


df_sales_rep = pd.DataFrame(sales_rep_data)
display(df_sales_rep.head())
display(df_sales_rep.tail())
print(f"Generated {len(df_sales_rep)} sales reps.")

,Rep_ID,Rep_Name,Email,Phone,Region_ID,Join_Date,Current_Status,Performance_Rating,Total_Deals_Closed,Avg_conversion_time,Win_rate
0,AS_NA,Alice Smith,as_na@crmcorp.com,+91-9838925472,NA,2022-01-03,Inactive,1.7,216,16.99,0.31
1,BJ_NA,Bob Johnson,bj_na@crmcorp.com,+91-9584350208,NA,2020-12-15,Inactive,3.8,387,21.69,0.70
2,DK_NA,Diana King,dk_na@crmcorp.com,+91-9978065083,NA,2021-10-17,On Leave,3.4,87,9.54,0.32
3,CL_NA,Charlie Lee,cl_na@crmcorp.com,+91-9728154151,NA,2021-02-07,Active,4.9,238,27.51,0.39
4,HB_EU,Hannah Black,hb_eu@crmcorp.com,+91-9568821149,EU,2020-06-03,On Leave,4.2,119,29.98,0.44


,Rep_ID,Rep_Name,Email,Phone,Region_ID,Join_Date,Current_Status,Performance_Rating,Total_Deals_Closed,Avg_conversion_time,Win_rate
15,PY_LA,Paula Young,py_la@crmcorp.com,+91-9941803624,LA,2021-01-05,On Leave,4.2,297,16.76,0.58
16,RA_LA,Rachel Allen,ra_la@crmcorp.com,+91-9702820483,LA,2021-05-02,Active,2.1,337,29.24,0.26
17,QH_LA,Quinn Hall,qh_la@crmcorp.com,+91-9792240592,LA,2022-09-14,On Leave,3.4,68,28.51,0.48
18,TB_AF,Tina Baker,tb_af@crmcorp.com,+91-9501948044,AF,2020-07-05,Active,4.7,279,16.95,0.90
19,SW_AF,Steve Wright,sw_af@crmcorp.com,+91-9824580672,AF,2020-01-17,On Leave,4.3,198,5.04,0.68


Generated 20 sales reps.


In [423]:
#COMPANIES
import pandas as pd
import numpy as np
import random

industries = ["Technology", "Healthcare", "Finance", "Manufacturing", "Retail", "E-Commerce", "Green Tech", "Mobility", "Real Estate", "Banking", "Logistics", "Housing", "Energy"] # Expanded industries
sizes = ["Start-up", "Small business", "Medium Enterprise", "Large Enterprise"]
ownerships = ["Public Sector", "Private Sector", "Joint Venture", "Franchise"]
funding = ["Bootstrapped", "Seed", "Series A", "Series B", "IPO"]
interest_areas = ["Data", "Marketing", "Cloud", "Design", "HR", "AI", "Consulting", "Engineering", "Operations", "Staffing", "T&M"] # Expanded interest areas

company_data = []
added_company_ids = set()

# Add companies from df_lead
for index, row in df_lead.iterrows():
    company_id = row['Company_ID']
    company_name = row['Company_Name']
    country = row['Country']
    # Add company data only if not already added
    if company_id not in added_company_ids:
        # Attempt to get industry and service from region_mapping if available, otherwise use random
        # Find the country info in the region_mapping dictionary
        current_region_data = next((data for region, data in region_mapping.items() if country in data["countries"]), None)

        company_industry = random.choice(current_region_data.get('industries', industries)) if current_region_data and current_region_data.get('industries') else random.choice(industries)
        interest_area = random.choice(current_region_data.get('services', interest_areas)) if current_region_data and current_region_data.get('services') else random.choice(interest_areas)

        company_data.append({
            "Company_ID": company_id,
            "Company_Name": company_name,
            "Company_Industry": company_industry, # Use industry from region_mapping or random
            "Company_Size": random.choice(sizes),
            "Company_Ownership": random.choice(ownerships),
            "Company_Funding": random.choice(funding),
            "Country": country, # Use country from df_lead
            "Interest_Area": interest_area # Use service from region_mapping or random
        })
        added_company_ids.add(company_id)

# Generate additional companies if needed (optional, based on whether df_lead has all companies)
# If df_lead contains all the companies you need for df_company, you might not need this part.
# Assuming df_lead has the comprehensive list of companies you want in df_company.
# (No additional generation needed as df_lead now contains 10000 companies)

df_company = pd.DataFrame(company_data)
display(df_company.head())
display(df_company.tail())
print(f"Generated {len(df_company)} companies.")

,Company_ID,Company_Name,Company_Industry,Company_Size,Company_Ownership,Company_Funding,Country,Interest_Area
0,CMP_U,Uber,Technology,Medium Enterprise,Franchise,IPO,USA,Cloud
1,CMP_W,Walmart,Technology,Start-up,Joint Venture,Seed,USA,Data
2,CMP_FMC,Ford Motor Co,Healthcare,Medium Enterprise,Public Sector,Bootstrapped,USA,Marketing
3,CMP_A,Amazon,Technology,Small business,Private Sector,Series B,USA,Data
4,CMP_M,Microsoft,Technology,Small business,Franchise,Series A,USA,Data


,Company_ID,Company_Name,Company_Industry,Company_Size,Company_Ownership,Company_Funding,Country,Interest_Area
9958,CMP_I_9888,InnovativeSystems,Finance,Small business,Private Sector,Seed,Mozambique,Service
9959,CMP_F_9889,FutureTech,Automotive,Start-up,Joint Venture,IPO,Israel,Engineering
9960,CMP_F_9890,FutureTech,Healthcare,Large Enterprise,Public Sector,IPO,USA,Cloud
9961,CMP_I_9891,InnovativeTech,Real Estate,Medium Enterprise,Franchise,Series A,UAE,Consulting
9962,CMP_C_9892,CreativeVentures,Mining,Large Enterprise,Public Sector,Seed,Morocco,Staffing


Generated 9963 companies.


In [424]:
#STAGE HISTORY
import pandas as pd
import random
from datetime import datetime, timedelta

# Define stages in the desired funnel order based on user provided ranges
funnel_stages = [
    "Lead Captured",
    "Qualified",
    "Contacted",
    "Responded",
    "Demo Scheduled",
    "Demo",
    "Demo Completed",
    "Proposal Sent",
    "Proposal",
    "Negotiation",
    "Deal Won",
    "Deal Lost"
]

stage_history_data = []
history_id = 1

# Define realistic time ranges for progression between stages (in days)
# Using the specific ranges provided by the user
stage_time_ranges = {
    "Lead Captured": (2, 7),        # 2–7 days to qualification
    "Qualified": (4, 9),            # 4–9 days to contact
    "Contacted": (5, 12),           # 5–12 days to responded/demo
    "Responded": (3, 6),            # 3–6 days to demo scheduled
    "Demo Scheduled": (4, 8),       # 4–8 days to demo
    "Demo": (2, 3),                 # 2–3 days to complete demo
    "Demo Completed": (4, 9),       # 4–9 days to proposal
    "Proposal Sent": (5, 11),       # 5–11 days to proposal
    "Proposal": (6, 13),            # 6–13 days to negotiation
    "Negotiation": (7, 15)          # 7–15 days to close
}


for index, lead in df_lead.iterrows():
    lead_id = lead.Lead_ID
    final_stage = lead['Current_Stage']

    # Ensure the final_stage is in funnel_stages before proceeding
    if final_stage not in funnel_stages:
        print(f"Warning: Lead {lead.Lead_ID} has a final stage '{final_stage}' not in funnel_stages. Skipping.")
        continue # Skip this lead if the final stage is not recognized

    # Filter the funnel stages to include only stages up to and including the final stage
    lead_stages_in_funnel = [stage for stage in funnel_stages if funnel_stages.index(stage) <= funnel_stages.index(final_stage)]

    # Ensure the stages follow the strict funnel order up to the final stage
    ordered_stages = lead_stages_in_funnel

    start_date = datetime.strptime(lead.Created_Date, "%Y-%m-%d %H:%M:%S")
    last_updated_date = datetime.strptime(lead.Last_Updated, "%Y-%m-%d %H:%M:%S")
    current_timestamp = start_date

    for seq, stage in enumerate(ordered_stages, start=1):
        timestamp = current_timestamp

        # For stages other than the first, add a time increment
        if seq > 1:
            prev_stage = ordered_stages[seq-2] # Get the name of the previous stage
            # Find the appropriate time range for the transition from the previous stage to the current stage
            # If a specific transition time isn't defined, use a default range (although all transitions are defined now)
            min_days, max_days = stage_time_ranges.get(prev_stage, (1, 7)) # Default to 1-7 days if range not found

            # Calculate available time for remaining stages
            time_remaining_seconds = (last_updated_date - current_timestamp).total_seconds()
            remaining_stages_count = len(ordered_stages) - seq

            # Ensure a minimum time increment of at least 1 second to prevent identical timestamps
            min_increment_seconds = 1
            # Ensure minimum days from stage_time_ranges are respected, converted to seconds
            min_days_seconds = min_days * 24 * 3600

            # Calculate the maximum possible increment to not exceed last_updated_date,
            # while leaving at least a minimal gap for subsequent stages if any remain.
            max_possible_increment_seconds = time_remaining_seconds
            if remaining_stages_count > 0:
                 max_possible_increment_seconds = time_remaining_seconds - (remaining_stages_count * 60) # Leave at least 1 minute for each subsequent stage


            # Determine the actual bounds for random time increment
            lower_bound_seconds = max(min_increment_seconds, min_days_seconds)
            upper_bound_seconds = min(max_days * 24 * 3600, max_possible_increment_seconds)

            # Ensure the range for random.uniform is valid. If upper_bound is less than lower_bound,
            # it means there's not enough time to meet the minimum realistic gap for remaining stages.
            # In such cases, distribute the remaining time as evenly as possible with a minimum 1-second gap.
            if upper_bound_seconds < lower_bound_seconds:
                if remaining_stages_count > 0:
                    time_increment_seconds = max(time_remaining_seconds / remaining_stages_count, 1) # Distribute remaining time, min 1 second gap
                else:
                    time_increment_seconds = max(time_remaining_seconds, 1) # If last stage, use remaining time, min 1 second
            else:
                 # Generate a random increment within the calculated realistic and possible bounds
                 time_increment_seconds = random.uniform(lower_bound_seconds, upper_bound_seconds)


            time_increment = timedelta(seconds=time_increment_seconds)
            timestamp = current_timestamp + time_increment

            # Final check to ensure timestamp is strictly increasing and not in the future
            if timestamp <= current_timestamp:
                 timestamp = current_timestamp + timedelta(seconds=random.randint(1, 60)) # Add a small increment if needed

            timestamp = min(timestamp, datetime.now()) # Ensure timestamp is not in the future

            # If it's the last stage, ensure its timestamp is exactly last_updated_date,
            # but still ensure it's not before the previous timestamp.
            if seq == len(ordered_stages):
                 timestamp = last_updated_date
                 if timestamp <= current_timestamp:
                      # If last_updated_date is before or same as previous timestamp,
                      # set it to previous timestamp + a small increment.
                      timestamp = current_timestamp + timedelta(seconds=random.randint(1, 60))


        else:
            # First stage timestamp is the Created_Date timestamp
            timestamp = start_date


        stage_history_data.append({
            "History_ID": history_id,
            "Lead_ID": lead_id,
            "Stage": stage,
            "Timestamp": timestamp, # Store as datetime for now
            "Rep_ID": lead.Rep_ID,
            "Stage_Sequence": seq,
            # Removed Is_Rentry column as requested
            "Notes": f"Notes for stage {stage} for Lead {lead_id}"
        })
        history_id += 1
        current_timestamp = timestamp # Update current_timestamp for the next iteration


df_stage_history = pd.DataFrame(stage_history_data)

# Ensure 'Timestamp' is datetime objects (should be already, but good practice)
df_stage_history['Timestamp'] = pd.to_datetime(df_stage_history['Timestamp'])

# Sort by Lead_ID and Stage_Sequence for correct time difference calculation and display order
df_stage_history = df_stage_history.sort_values(by=['Lead_ID', 'Stage_Sequence'])

# Calculate Stage_Engagement (rounded to nearest day and as a number)
stage_engagement_days = []

for lead_id, group in df_stage_history.groupby('Lead_ID'):
    group = group.reset_index(drop=True)
    for i in range(len(group)):
        if i == 0:
            # First stage entry for the lead
            days = 0 # Engagement at the point of capture is 0
        else:
            # For all subsequent stages, calculate difference with the previous stage
            time_diff = group.iloc[i]['Timestamp'] - group.iloc[i-1]['Timestamp']
            # Round to the nearest day, ensure it's not negative
            days = round(time_diff.total_seconds() / (24 * 3600))
            days = max(days, 0) # Ensure non-negative

        stage_engagement_days.append(days)

# Add the calculated 'Stage_Engagement' column with the new name and numerical data
df_stage_history['Stage_Engagement (in Days)'] = stage_engagement_days

# Remove the old 'Stage_Engagement' column if it exists
if 'Stage_Engagement' in df_stage_history.columns:
    df_stage_history = df_stage_history.drop(columns=['Stage_Engagement'])

# Remove the 'Is_Cold' column if it exists (based on previous requests)
if 'Is_Cold' in df_stage_history.columns:
    df_stage_history = df_stage_history.drop(columns=['Is_Cold'])

# Display samples of the generated DataFrame
display(df_stage_history.head())
display(df_stage_history.tail())

print(f"Generated {len(df_stage_history)} stage history entries.")

# After generating the df_stage_history, update the 'Current_Stage' in df_lead
# to reflect the latest stage from the generated df_stage_history.
# This part is moved here to ensure df_stage_history is available.

# Find the latest stage for each lead from df_stage_history (based on maximum Stage_Sequence)
# Ensure df_stage_history is sorted by Lead_ID and Timestamp first to correctly identify the latest stage
df_stage_history = df_stage_history.sort_values(by=['Lead_ID', 'Timestamp'])

# Get the index of the last stage entry for each lead
latest_stage_indices = df_stage_history.groupby('Lead_ID')['Timestamp'].idxmax()

# Select the rows corresponding to the latest stages
latest_stages = df_stage_history.loc[latest_stage_indices][['Lead_ID', 'Stage']]
latest_stages.rename(columns={'Stage': 'Actual_Current_Stage'}, inplace=True)

# Merge with df_lead to update 'Current_Stage'
# Use a left merge to keep all leads in df_lead
df_lead = df_lead.merge(latest_stages, on='Lead_ID', how='left', suffixes=('', '_latest_history'))

# Update 'Current_Stage' in df_lead with the 'Actual_Current_Stage' from history if available
# If a lead exists in df_lead but not in df_stage_history (shouldn't happen with current logic but as a safeguard)
# or if the merge didn't find a match for some reason, keep the original 'Current_Stage'.
df_lead['Current_Stage'] = df_lead['Actual_Current_Stage'].fillna(df_lead['Current_Stage'])


# Drop the temporary 'Actual_Current_Stage' column
df_lead = df_lead.drop(columns=['Actual_Current_Stage'])

# Display samples of the updated df_lead
print("\nUpdated df_lead with latest stage information:")
display(df_lead.head())
display(df_lead.tail())

,History_ID,Lead_ID,Stage,Timestamp,Rep_ID,Stage_Sequence,Notes,Stage_Engagement (in Days)
203,204,LDABB-EU-032,Lead Captured,2024-06-25 08:27:39.000000,HB_EU,1,Notes for stage Lead Captured for Lead LDABB-E...,0
204,205,LDABB-EU-032,Qualified,2024-06-27 11:48:28.584780,HB_EU,2,Notes for stage Qualified for Lead LDABB-EU-032,2
205,206,LDABB-EU-032,Contacted,2024-06-27 13:21:06.667824,HB_EU,3,Notes for stage Contacted for Lead LDABB-EU-032,0
206,207,LDABB-EU-032,Responded,2024-06-27 14:53:44.750868,HB_EU,4,Notes for stage Responded for Lead LDABB-EU-032,0
207,208,LDABB-EU-032,Demo Scheduled,2024-06-27 16:26:22.833912,HB_EU,5,Notes for stage Demo Scheduled for Lead LDABB-...,0


,History_ID,Lead_ID,Stage,Timestamp,Rep_ID,Stage_Sequence,Notes,Stage_Engagement (in Days)
577,578,LDZNR-APAC-087,Responded,2024-09-28 16:35:16.279716,JL_APAC,4,Notes for stage Responded for Lead LDZNR-APAC-087,0
578,579,LDZNR-APAC-087,Demo Scheduled,2024-09-28 17:14:40.186477,JL_APAC,5,Notes for stage Demo Scheduled for Lead LDZNR-...,0
579,580,LDZNR-APAC-087,Demo,2024-09-28 17:54:04.093238,JL_APAC,6,Notes for stage Demo for Lead LDZNR-APAC-087,0
580,581,LDZNR-APAC-087,Demo Completed,2024-09-28 18:33:28.000000,JL_APAC,7,Notes for stage Demo Completed for Lead LDZNR-...,0
581,582,LDZNR-APAC-087,Proposal Sent,2024-09-28 18:33:30.000000,JL_APAC,8,Notes for stage Proposal Sent for Lead LDZNR-A...,0


Generated 65262 stage history entries.

Updated df_lead with latest stage information:


,Lead_ID,Lead_Name,Company_Name,Source,Region_ID,Rep_ID,Created_Date,Current_Stage,Email,Phone,...,Last_Updated,Deal_Source,Company_ID,Engagement_Level,Follow_Up_Threshold,Project_Term,Budget_Tier,Country,Company_Industry,Service_Category
0,LDUBR-NA-001,Uber Lead,Uber,Outbound,NA,AS_NA,2024-07-30 17:33:27,Demo Scheduled,contact_70@uber.com,+1-2237976190,...,2024-08-06 13:00:27,Outbound,CMP_U,Medium,7,6-12 months,50-100L,USA,Retail,Marketing
1,LDWMT-NA-002,Walmart Lead,Walmart,Outbound,NA,BJ_NA,2024-11-14 20:50:13,Demo Scheduled,contact_47@walmart.com,+1-8466238837,...,2024-11-15 17:11:13,Outbound,CMP_W,Medium,3,6-12 months,20-50L,USA,Technology,Marketing
2,LDFRD-NA-003,Ford Motor Co Lead,Ford Motor Co,Outbound,NA,DK_NA,2023-11-16 17:05:59,Deal Lost,contact_61@fordmotorco.com,+1-1657048373,...,2023-11-17 08:53:59,Outbound,CMP_FMC,Low,5,12-24 months,20-50L,USA,Healthcare,Cloud
3,LDAMZ-NA-004,Amazon Lead,Amazon,Outbound,NA,BJ_NA,2024-08-14 06:24:18,Contacted,contact_15@amazon.com,+1-9338127286,...,2024-08-15 14:55:18,Outbound,CMP_A,High,5,>24 months,100L+,USA,Retail,Cloud
4,LDMST-NA-005,Microsoft Lead,Microsoft,Outbound,NA,CL_NA,2024-10-01 11:37:37,Demo Scheduled,contact_58@microsoft.com,+1-4400837397,...,2024-10-02 17:29:37,Outbound,CMP_M,High,3,6-12 months,50-100L,USA,Healthcare,Cloud


,Lead_ID,Lead_Name,Company_Name,Source,Region_ID,Rep_ID,Created_Date,Current_Stage,Email,Phone,...,Last_Updated,Deal_Source,Company_ID,Engagement_Level,Follow_Up_Threshold,Project_Term,Budget_Tier,Country,Company_Industry,Service_Category
9995,LDINNO-AF-9888,InnovativeSystems Lead,InnovativeSystems,Inbound,AF,SW_AF,2024-10-14 11:03:02,Negotiation,contact_69@innovativesystems.com,+258-5631770211,...,2024-10-16 05:24:02,Inbound,CMP_I_9888,High,7,0-6 months,50-100L,Mozambique,Telecommunications,Staffing
9996,LDFUTU-EU-9889,FutureTech Lead,FutureTech,Outbound,EU,GW_EU,2024-08-04 12:34:07,Proposal Sent,contact_92@futuretech.com,+972-1347982024,...,2024-08-08 23:41:07,Outbound,CMP_F_9889,High,10,>24 months,100L+,Israel,Consulting,Operations
9997,LDFUTU-NA-9890,FutureTech Lead,FutureTech,Outbound,NA,BJ_NA,2023-10-10 01:11:53,Proposal Sent,contact_33@futuretech.com,+1-2723807852,...,2023-10-11 23:58:53,Outbound,CMP_F_9890,High,5,6-12 months,50-100L,USA,Healthcare,Marketing
9998,LDINNO-ME-9891,InnovativeTech Lead,InnovativeTech,Outbound,ME,OC_ME,2025-07-09 08:09:08,Negotiation,contact_29@innovativetech.com,+971-6262904144,...,2025-07-11 15:02:08,Outbound,CMP_I_9891,Low,10,0-6 months,20-50L,UAE,Banking,Data
9999,LDCREA-AF-9892,CreativeVentures Lead,CreativeVentures,Outbound,AF,TB_AF,2024-08-12 08:58:32,Lead Captured,contact_49@creativeventures.com,+212-4618477631,...,2024-08-12 10:20:32,Outbound,CMP_C_9892,Low,7,6-12 months,20-50L,Morocco,Finance,Staffing


In [425]:
# DEALS
import pandas as pd
import random
from datetime import datetime, timedelta

def generate_deals(df_lead):
    deals_data = []
    deal_id_counter = 1

    # Filter for leads that reached 'Deal Won' or 'Deal Lost' stages
    closed_leads = df_lead[df_lead['Current_Stage'].isin(['Deal Won', 'Deal Lost'])]

    for index, lead in closed_leads.iterrows():
        # Find the 'Deal Won' or 'Deal Lost' timestamp from stage history
        closed_stage_history = df_stage_history[(df_stage_history['Lead_ID'] == lead['Lead_ID']) &
                                                 (df_stage_history['Stage'].isin(['Deal Won', 'Deal Lost']))]

        if not closed_stage_history.empty:
            # Assuming the latest 'Deal Won' or 'Deal Lost' entry is the closing date
            # The Timestamp column is already datetime objects, so no need for strptime
            closed_date = closed_stage_history.sort_values(by='Timestamp', ascending=False).iloc[0]['Timestamp'].date()

            # Ensure created date is before closed date
            # The Created_Date column is already datetime objects (from previous modifications), so no need for strptime
            created_date = pd.to_datetime(lead['Created_Date']).date() # Ensure Created_Date is datetime before accessing .date()


            # Ensure closed date is not before created date, adjust if necessary
            if closed_date < created_date:
                closed_date = created_date + timedelta(days=random.randint(1, 30)) # Add at least a day


            deals_data.append({
                "Deal_ID": f"DEAL_{deal_id_counter:04}",
                "Lead_ID": lead['Lead_ID'],
                "Company_ID": lead['Company_ID'],
                "Amount": round(random.uniform(500, 5000), 2), # Random deal amount
                "Status": lead['Current_Stage'], # Directly use 'Deal Won' or 'Deal Lost'
                "Created_Date": created_date.strftime("%Y-%m-%d"),
                "Closed_Date": closed_date.strftime("%Y-%m-%d"),
                "Rep_ID": lead['Rep_ID']
            })
            deal_id_counter += 1

    return pd.DataFrame(deals_data)

df_deals = generate_deals(df_lead)
display(df_deals.head())
display(df_deals.tail())
print(f"Generated {len(df_deals)} deals.")

# Add df_deals to the sheet_mapping
# sheet_mapping["Deals"] = df_deals # This line caused the error, moved sheet_mapping definition earlier

,Deal_ID,Lead_ID,Company_ID,Amount,Status,Created_Date,Closed_Date,Rep_ID
0,DEAL_0001,LDFRD-NA-003,CMP_FMC,2608.27,Deal Lost,2023-11-16,2023-11-17,DK_NA
1,DEAL_0002,LDMCK-NA-009,CMP_M,1547.27,Deal Lost,2025-05-22,2025-05-25,BJ_NA
2,DEAL_0003,LDKPR-NA-013,CMP_KP,3389.23,Deal Lost,2024-11-16,2024-11-21,DK_NA
3,DEAL_0004,LDWLF-NA-014,CMP_WF,3604.51,Deal Lost,2024-07-14,2024-07-17,AS_NA
4,DEAL_0005,LDTHC-NA-017,CMP_TH,4619.45,Deal Lost,2024-12-06,2024-12-12,CL_NA


,Deal_ID,Lead_ID,Company_ID,Amount,Status,Created_Date,Closed_Date,Rep_ID
2507,DEAL_2508,LDINNO-EU-9875,CMP_I_9875,2026.48,Deal Lost,2024-03-17,2024-03-20,HB_EU
2508,DEAL_2509,LDSTRA-APAC-9877,CMP_S_9877,2769.88,Deal Lost,2023-10-20,2023-10-22,IG_APAC
2509,DEAL_2510,LDINNO-APAC-9883,CMP_I_9883,1337.11,Deal Won,2024-01-13,2024-01-15,MT_APAC
2510,DEAL_2511,LDSTRA-EU-9884,CMP_S_9884,1125.05,Deal Won,2025-02-26,2025-03-03,EB_EU
2511,DEAL_2512,LDSTRA-EU-9886,CMP_S_9886,3451.10,Deal Won,2023-11-06,2023-11-11,GW_EU


Generated 2512 deals.


In [426]:
!pip install --upgrade gspread gspread_dataframe

import gspread
from gspread_dataframe import set_with_dataframe
from google.colab import auth
auth.authenticate_user()


In [427]:
# Use the spreadsheet name you created
spreadsheet = client.open("CRM_Capstone")

In [ ]:
# Sheet mappings
sheet_mapping = {
    "Leads": df_lead,
    "Sales_Reps": df_sales_rep,
    "Regions": df_region,
    "Stage_History": df_stage_history,
    "Deals":df_deals,
    "Companies": df_company
}

# Write to each sheet
for sheet_name, df in sheet_mapping.items():
    try:
        worksheet = spreadsheet.worksheet(sheet_name)
    except gspread.WorksheetNotFound:
        worksheet = spreadsheet.add_worksheet(title=sheet_name, rows=df.shape[0], cols=df.shape[1])

    worksheet.clear()
    set_with_dataframe(worksheet, df)

print("All DataFrames written to Google Sheets.")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
